# CSE213 - Numerical Analysis

# Lab 1 - Number Representation and Errors

### Python's Number Representation

The IEEE-754 standard defines a binary floating point format with a 64-bit (double precision) number.  The format is as follows:


- A 1-bit sign bit (0 for positive, 1 for negative).
- A 11-bit exponent (with a bias of 1023).
- A 52-bit mantissa.



The number is represented as $$(-1)^{\text{sign}} \times 2^{\text{exponent} - 1023} \times (1 + \sum_{k=0}^{51} b_k \times 2^{-k-1})\text{,}$$ where $b_k$ is the $k$th bit of the mantissa.

For example, the number $1.5$ is represented as $(-1)^{0} \times 2^{1023 - 1023} \times (1 + 1 \times 2^{-1} + 0 \times 2^{-2})$.



#### Machine Epsilon
The fact that the mantissa has 52 bits after the binary point means that the next machine number greater than 1 is $1 + 2^{-52}$.  This gap is called the **Machine Precision**, or **Machine Epsilon**. This and other key constants of Python’s arithmetic are easily obtained from the `sys` module (import `sys` ﬁrst).

```python
import sys 
print (sys.float_info.epsilon)
print(sys.float_info.min)
print(sys.float_info.max)
```

##### Exercise 1
Write python code to find the machine epsilon (without importing the `sys` module).

In [ ]:
def calculateEpsilon():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# TEST CASE
assert calculateEpsilon() == 2.220446049250313e-16

### Relative and Absolute Errors

##### Exercise 2
Verify--using Python code--the absolute and relative errors in approximating $e$ by $2.7183$. What are the corresponding errors in the approximation $100*e \approx 271.83$? In Python use `math.exp(1)` as the true value of $e$.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Problems Arising From Floating Point Arithmetic
Before we start, we'll use a straightforward floating-point representation to demonstrate. This representation can accommodate up to four meaningful digits (-9.999 to 9.999), exponents from -3 to 4, and base 10 for convenient manual verification.

Let's examine some of the issues arising from floating-point arithmetic: 
- Overflow
- Underflow
- Loss of precision when converting to floating-point
- Adding numbers of very different magnitudes
- Subtracting numbers of similar magnitudes
- Multiplying and dividing.

#### Overflow
When dealing with our simple system, numbers greater than 99,990 ($9.999 \times 10^4$) will cause an overflow. Overflow happens when the number you are trying to represent in floating point is greater than what can be expressed by the system. For our simple example, the largest allowable number is 99,990 ($9.999 \times 10^4$). Since numbers typically get rounded to the nearest number with the allowed significant digits, it would have to round to 100,000 ($10.0 \times 10^4$) for the number to exceed the system's capacity.

The way overflow is handled depends on the system, but most often it will be returned as infinity or not a number `NaN`. If the number is below $-9.999 \times 10^4$, a signed result like -infinity may be returned.

#### Underflow
When the number being represented is too small, it is called underflow. In this case, the smallest number representable is $0.001 \times 10^-3$, or $10^-6$. Anything lower than this, such as $10^-7$, would produce an underflow and be rounded to zero.

#### Loss Of Precision In Converting Into Floating Point
We can convert the number $1/3$ into our floating point system. The decimal is repeating: $0.333333\dots$, and our system writes it as $3.333 \times 10^-1$. How different is this from its real value? The difference is $0.0000333333\dots$ or $1/3 \times 10^-4$.

When we convert decimal numbers into binary, we lose precision. For example, $0.1$ and $0.4$ look correct in decimal, but in binary they are repeating decimals. So, when we convert $0.1$ and $0.4$ into binary and subtract them from $0.5$, we don't get $0$.

We can think of this in terms of significant figures and how the system sets a limit on how many figures we can have. This helps us understand how precision is lost in floating point operations.

#### Adding Numbers Of Very Different Magnitudes
Let's add $7.777 \times 10^2$ and $9.555 \times 10^-1$. This is like adding $777.7$ to $.9555$. The exact answer is $778.6555$, but too many decimal places for our system. So, we need to round it to $778.7$. The difference is $0.0445$. This is very small, so maybe it's ok.

But, if we are adding up a bunch of numbers to get one, and each number is very small, this can be a problem. For example, adding $0.010 \times 10^-3$ to $9.999 \times 10^-1$ gives $0.9999 + 0.00001= 0.99991 = 0.9999$ (strictly, it is $9.999 \times 10^-4$) in our system.

If we have this row of numbers:

$.9999$, $.00001$, $.00001$, $.00001$, $.00001$, $.00001$, $.00001$, $.00001$, $.00001$, $.00001$, $.00001$

It should equal one, but adding them step-by-step from left to right in pairs, each $0.010 \times 10^-3$ is like adding zero to $0.9999$.

To handle this, we should add the small numbers together first. We can add them from bottom to top or add them in pairs. Note that addition is not associative in floating point.

> **Tip 1**
>> Whenever possible, add numbers of similar small magnitude together before trying to add to larger magnitude numbers.

#### Subtracting Numbers Of Similar Magnitudes
The problem is losing digits when subtracting. For example, if we subtract $9.999$ and $9.998$, we get $0.001$ or $1.000 \times 10^-3$. We started with four digits, but only ended up with one.

This happens when we convert numbers from outside the floating point system (like from decimal to binary). Say the original numbers were $9.999$ and $9.997571$. The exact difference should be $0.001429$, which is $1.429 \times 10^-3$ in our system. But since $9.997571$ is not expressible, it would be rounded to $9.998$ and the difference would be $1.000 \times 10^-3$.

So what should we do?

Before converting, subtract $9.997$ from both numbers. That gives us $0.002 = 2.000 \times 10^-3$ and $0.000571 = 0.571 \times 10^-3$.

> **Tip 2**
>> Before subtracting floating point numbers, you may need to adjust the original numbers or modify the algorithm to avoid losing any significant figures during subtraction.

#### Multiplying and Dividing
The problem with multiplication and division is that of overflow and underflow.

A simple yet effective advice is to try to multiply and divide expressions as close to one as possible, to keep from underflow and overflow, and the loss of precision that results when you get close to these limits.

Using our system, suppose you wanted to do:

$(99990 \times 19990 \times 22220) / (11110 \times 77770)$

If we did it in a strictly left-to-right calculation, we’d overflow on our first operation. If we calculated it like this:

$1/77770 \times 1/11110 \times (99990 \times 199990 \times 22220)$

We would underflow on the first multiplication of the two reciprocals.

What you would want to do is rearrange the steps (there is more than one way of doing this arrangement to prevent problems):

$(99990/77770) \times (22220/11110) \times 19990 = 1.286 \times 2.000 \times 19990 = 5.141 \times 10^4$ (in our system)

> **Tip 3**
>> To prevent overflow and underflow (as well as loss of precision) when multiplying and dividing numbers, try to rearrange the product so that one is multiplying by expressions near to one.

#### Exercises

##### Exercise 3
Implement the algorithm, given below, in Python and check what values of `x` will be printed? Explain what happened.
```
x = 0.0;
while x not equal 1.0
    print x
    x = x + 0.1;
``` 

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Complete your answer here if your exercise requires comments!

##### Exercise 4
The following Python program shows the effect of the propagation of an initial representation error which results in ﬁnal (large!) error. Run the program, print its results once for `k` in `range(10)` and in `range(30)`. Comment on the obtained results.

In [ ]:
for n in range(1,11):
    a = []
    x = 1 / n
    for k in range(30):
        x = (1 + n) * x - 1
        a.append(x)
    print(a)
    print()

Complete your answer here if your exercise requires comments!

##### Exercise 5
Implement the problem for your specific question model.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Complete your answer here if your exercise requires comments!

### Extra Notes

**1**. In most cases we wish to use the `NumPy` package for Python for numerical computations. `NumPy` deﬁnes its own ﬂoating-point and integer data types offering choice between different levels of precision, e.g. `numpy.float16`, `numpy.float32`, and `numpy.float64` (double precision is the default type). 

In Python, neither underﬂow nor overﬂow cause a “program” to stop. Underﬂow is replaced by zero, while overﬂow is replaced by $\pm \infty$. This allows subsequent instructions to be executed and may permit meaningful results. Frequently, however, it will result in meaningless answers such as $\pm \infty$ or `NaN`, which stands for Not a-Number. `NaN` is the result of indeterminate arithmetic operations such as $0/0$, $\infty / \infty$, $0 \times \infty$, $\infty - \infty$, etc.

**2**. The exponents $\mathtt{000}_{16}$ and $\mathtt{7FF}_{16}$ have a special meaning:

- $\mathtt{000}_{16}$ is used to represent a signed zero (if $F = 0$) and subnormal numbers (if $F \neq 0$); and
- $\mathtt{7FF}_{16}$ is used to represent $\infty$ (if $F = 0$) and NaN (if $F \neq 0$),

where $F$ is the fractional part of the mantissa.

**3**. Some Python implementations use the IEEE-754 standard with round-to-nearest-ties-to-even rounding.

In the IEEE-754 double precision, the $1 + 2^{-53}$ is the midpoint between $1$ and the next representable number, $1 + 2^{-52}$. Since this is a tie, the round-to-even rule applies. The significand for $1$ is $1.000\dots000_2$, and the significand for $1 + 2^{-52}$ is $1.000\dots001_2$. Since the former is even, the round-to-even tie-breaking rule chooses it, and the result is $1$.

Consider instead $1 + 3 \times 2^{-54}$. This is three-quarters of the way from $1$ to $1 + 2^{-52}$. So rounding it to the nearest representable value will produce $1 + 2^{-52}$. For the expression `print(1 == 1+3*2**-54)`, Python implementation will likely print "`False`".

# **Assignment Solution**
Name: *Assem Esmail*
ID: *120210321*
Date: *20/2/2023*

In [1]:

check_input = True #Check if user input is valid
while check_input: #Loop until user input is valid
    a, b, c = eval(input("Please enter the a, b, c coefficients of your quadratic equation: ")) #Get user input for equation coefficients (a , b, c) 
    try: 
        float(a), float(b), float(c) 
        check_input = False 
    except ValueError: 
        print("Please make sure the coefficients are real numbers and try again") 
        check_input = True 
   #Calculate discriminant using Python and Finding roots of equation     
discriminant = b**2 - 4*a*c #Calculate discriminant
if discriminant < 0: #Check if discriminant is negative
    print("There are no real solutions") #If discriminant is negative, there are no real solutions
elif discriminant == 0: #Check if discriminant is zero
    print("There is one solution: {}".format(-b/(2*a))) #If discriminant is zero, there is one solution (x = -b/2a)
else: #If discriminant is positive, there are two solutions
    print("There are two solutions: {} and {}".format((-b + discriminant**0.5)/(2*a), (-b - discriminant**0.5)/(2*a))) #Calculate and print solutions

Please enter the a, b, c coefficients of your quadratic equation: 1, 62.10, 1
There are two solutions: -0.016107237408967023 and -62.083892762591034


### References
1. “[Double-precision floating-point format](https://en.wikipedia.org/wiki/Double-precision_floating-point_format),” Wikipedia
2. [Losing My Precision: Tips For Handling Tricky Floating Point Arithmetic](https://www.soa.org/news-and-publications/newsletters/compact/2014/may/com-2014-iss51/losing-my-precision-tips-for-handling-tricky-floating-point-arithmetic/) | SOA. (n.d.-b). 
3. Burden, R. L., & Faires, D. J. (2010). Numerical Analysis (9th ed.). Cengage Learning.